In [ ]:
!ls /path/
# http://www.apache.org/licenses/LICENSE-2.0

In [ ]:
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
data_path = 'path'

In [ ]:
size = 128
x_train, y_train = [], []
x_test, y_test = [], []
x_val, y_val = [], []

for data_split in ['train', 'test', 'val']:
    split_path = data_path + data_split + '/'

    for label in ['NORMAL', 'PNEUMONIA']:
        label_path = split_path + label + '/'

        image_files = glob.glob(label_path + '*.jpeg')

        for image_file in image_files:
            image = cv2.imread(image_file)
            image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)
            if data_split == 'train':
                x_train.append(image)
                y_train.append(0 if label == 'NORMAL' else 1)
            elif data_split == 'test':
                x_test.append(image)
                y_test.append(0 if label == 'NORMAL' else 1)
            else:
                x_val.append(image)
                y_val.append(0 if label == 'NORMAL' else 1)

In [ ]:

x_train = np.array(x_train, dtype = 'float32')
x_test = np.array(x_test, dtype = 'float32')
x_val = np.array(x_val, dtype = 'float32')
y_train = np.array(y_train, dtype = 'int32')
y_test = np.array(y_test, dtype = 'int32')
y_val = np.array(y_val, dtype = 'int32')


x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0
x_val_scaled = x_val / 255.0

print(x_train.shape)

In [ ]:
#CNN model
inp = Input(shape=(128,128,3))
cnn = Conv2D(filters=8, kernel_size = 3, activation='relu')(inp)
pooling = MaxPooling2D(pool_size=(2,2))(cnn)
drop = Dropout(0.2)(pooling)

cnn = Conv2D(filters=16, kernel_size = 4, activation='relu')(drop)
cnn = BatchNormalization()(cnn)
pooling = MaxPooling2D(pool_size=(2,2))(cnn)
drop = Dropout(0.2)(pooling)

cnn = Conv2D(filters=32, kernel_size = 4, activation='relu')(drop)
pooling = MaxPooling2D(pool_size=(2,2))(cnn)

f = Flatten()(pooling)

fc1 = Dense(units=256, activation='relu') (f)
fc2 = Dense(units=16, activation='relu') (fc1)
out = Dense(units=1, activation='sigmoid') (fc2)

model = Model(inputs = inp, outputs = out)
model.summary()

In [ ]:
from IPython.core import history
optimizer1 = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer1, loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train_scaled, y_train, batch_size=64, epochs=30,
                    validation_data=(x_test_scaled, y_test))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('model2.h5')

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('/kaggle/working/model2.h5')

In [ ]:
y_guess = model1.predict(x_test_scaled)


In [ ]:
def predict(y_guess):
    for i in range(len(y_guess)):
        y_guess[i] = 0 if y_guess[i] < 0.5 else 1
    return y_guess

In [ ]:
y_pre = predict(y_guess)

In [ ]:
import sklearn
import seaborn as sns


cm = sklearn.metrics.confusion_matrix(y_test, y_pre)


plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pre)
precision = precision_score(y_test, y_pre)
recall = recall_score(y_test, y_pre)
f1 =f1_score(y_test, y_pre)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:

model_path = 'path'


loaded_model = tensorflow.keras.models.load_model(model_path)

In [ ]:
model = Sequential()

model.add(loaded_model)

for layer in loaded_model.layers:
    layer.trainable = False

model.add(Flatten())

model.add(Dense(units = 256, activation ='relu'))
model.add(Dense(units = 16, activation ='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(units = 1, activation = 'sigmoid'))
model.summary()


In [ ]:
from IPython.core import history
optimizer1 = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer1, loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train_scaled, y_train, batch_size=64, epochs=30,
                    validation_data=(x_test_scaled, y_test))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('pretrain_1.h5')


In [ ]:
y_guess2 = model.predict(x_test_scaled)


In [ ]:
y_pre2 = predict(y_guess2)


In [ ]:
import sklearn
import seaborn as sns


# Tạo confusion matrix
cm = sklearn.metrics.confusion_matrix(y_test, y_pre2)

# Vẽ confusion matrix
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pre2)
precision = precision_score(y_test, y_pre2)
recall = recall_score(y_test, y_pre2)
f1 =f1_score(y_test, y_pre2)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")